- enviroment


In [15]:
import gensim
from scipy import spatial
import json
import numpy as np
import nltk
from gensim.models import Word2Vec

- data file

In [2]:
train_set = "/Users/alfredchen/Documents/GitHub/query-system/data/training.json"
dev_set = "/Users/alfredchen/Documents/GitHub/query-system/data/devel.json"
test_set = "/Users/alfredchen/Documents/GitHub/query-system/data/testing.json"
doc = "/Users/alfredchen/Documents/GitHub/query-system/data/documents.json"

#save word embeddings
embeddings = "/Users/alfredchen/Documents/GitHub/query-system/models/mymodel-size"

# save qa log
log = "/Users/alfredchen/Documents/GitHub/query-system/data/log.txt"

- preprocess corpus
    - remove stopwords
    - lemmatize
    - lower case
    - creat paragraph_index
    - create index2paragraph

In [3]:
    def get_stopwords():
        
        words = list(nltk.corpus.stopwords.words())
        
        #some stopwords are helpful in targeting answer type, shall not be removed
        words.remove('what')
        words.remove('where')
        words.remove('when')
        words.remove('who')
        words.remove('how')
        words.remove('which')
        # add more #

        stopwords = {}
        for word in words:
            stopwords[word] = stopwords.get(word,0) + 1
        return stopwords

In [4]:
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word, 'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word, 'n')
    return lemma


stopwords = get_stopwords()        # get stopwords

def preprocess_docs(corpus):
        para2index = {}
        index2para = {}
        new_corpus = []
        for _id, doc in enumerate(corpus):
            new_doc = []
            for _para,para in enumerate(doc['text']):
                para2index[para] = (_id,_para)         # para index
                index2para[(_id,_para)] = para         # doc_id, answer_para index
                new_para = []
                sents = para.split('.')
                for _sent,sent in enumerate(sents):
                    new_sent=[]
                    sent = '<s> ' + sent + ' <end>'  # padding
                    words = sent.split(' ')
                    s = ''.join(sent)
                    if s == '<s>  <end>':
                        continue
                    for word in words:
                        word = word.strip(',')
                            #if not word.isalpha():
                                #continue
                        new_word = word.lower()
                        if stopwords.get(new_word):
                            continue
                        new_word = lemmatize(new_word)
                        new_sent.append(new_word)                
                    new_para.append(new_sent)
                new_doc.append(new_para)
            new_corpus.append(new_doc)
        return new_corpus, para2index, index2para

In [5]:
docs = json.load(open(doc))
corpus,para2index,index2para = preprocess_docs(docs)

- get word embeddings based on pre processed corpus

In [24]:
    def w2v(corpora,size,iter):
        docs = corpora
        sentences = []
        for doc in docs:
            for sents in doc:
                new_para = []
                for sent in sents:
                    new_para += sent
                sentences.append(new_para)
        model = Word2Vec(sentences, size=size, iter=iter)
        model.save(embeddings+str(size)+'-iter'+str(iter))
        return model

In [25]:
model = w2v(corpus,100,200)   # train the model

#load pre-trained model
#model = gensim.models.Word2Vec.load(embeddings+str(300)+'-iter'+str(100))

- transform sentence to feature vector (sentence embeddings)

In [28]:
    index2word_set = set(model.wv.index2word)
    
    
    
    def sent2vec(sentence, model, size, index2word_set):
        """transform word embeddings to sentence vector
        param: sentence: sentence that want to be transformed
        param: model: pre-trained word embeddings
        param: size: feature vector dimension
        param: index2word_set
        return: transformed sentence vector
        """
        try:
            words = sentence.split()
        except:
            words = sentence
        feature_vec = np.zeros((size,), dtype='float32')
        n_words = 0
        for word in words:
            word = word.lower()
            if stopwords.get(word):
                continue
            if word in index2word_set:
                n_words += 1
                feature_vec = np.add(feature_vec, model.wv[word])
        if (n_words > 0):
            feature_vec = np.divide(feature_vec, n_words)
        return feature_vec

- play with sentence vector

In [30]:
# calculate similarity of two sentence

s1_afv = sent2vec('what is the capital of china', model, 100, index2word_set)
s2_afv = sent2vec('the capital of china is beijing', model, 100, index2word_set)
sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
print(sim)

0.785611629486084


In [31]:
#print(index2para.get((4,1)))     #get the first paragraph in 4th document

- get similarity ranking of query to paragraphs in a documents

In [34]:
def sim_sent(query, corpus):
    sim2ipara = {}
    index2sim = {}
    query = sent2vec(query, model, 100, index2word_set)
    for _id,doc in enumerate(corpus):
        for _para,para in enumerate(doc):
            sentence = []
            for sent in para:
                sentence += sent
            sentence = sent2vec(sentence, model, 100, index2word_set)
            sim = 1 - spatial.distance.cosine(query, sentence)
            index2sim[_id, _para] = sim
            sim2ipara[sim] = (_id,_para)
    return index2sim, sim2ipara
    

#index2sim, sim2ipara = sim_sent("what does the Planck constant refer to?",corpus)



- get top n answer paragraph given quey

In [38]:
def getTopN(N,query):
    index2sim, sim2ipara = sim_sent(query,corpus)
    similarity_ranks = sorted(index2sim.values(),reverse=True)   #sims
    rank = []
    for i in range(N):
        rank.append(sim2ipara.get(similarity_ranks[i])) 
    return rank

getTopN(10,"what does plank constant refer do")  



[(358, 4),
 (439, 0),
 (57, 1),
 (282, 4),
 (191, 32),
 (345, 1),
 (127, 0),
 (187, 23),
 (385, 1),
 (57, 19)]

In [39]:
#play: what is the capital of Australia

index2para.get((358,4))



'In 1901, the Federation of Australia was the process by which the six separate British self-governing colonies of New South Wales, Queensland, South Australia, Tasmania, Victoria and Western Australia formed one nation. They kept the systems of government that they had developed as separate colonies but also would have a federal government that was responsible for matters concerning the whole nation. When the Constitution of Australia came into force, the colonies collectively became states of the Commonwealth of Australia.'

- test hit accuracy: return top 10 paragraphs, is given answer included?

In [58]:
def gold_standard(dataset):
    file = json.load(open(dataset))
    gold2index={}
    for line in file:
        _id = line['docid']
        try:
            _para = line['answer_paragraph']
        except KeyError:
            _para = line['id']
        gold2index[line['question']]=(_id,_para)
    return gold2index

def my_qa(n,dataset):
    file = json.load(open(dataset))
    qa2index={}
    for i in range(n):
        save = open(log,"a")
        line = file[i]
        query = line['question']
        possible = getTopN(20,query)
        qa2index[line['question']] = possible
        record = line['question']+':'+str(possible)+'\n'
        save.write(record)
        save.close()
    return qa2index
    
    

In [54]:
def acc(gold_standard,qas):
    query = qas.keys()
    total = 0
    correct = 0
    for i,q in enumerate(query):
        gold = gold_standard.get(q)
        if gold in qas.get(q):
            correct += 1
        total = i
    acc = correct/total
    return acc

In [45]:
#test on top 100 trainset
gold_index = gold_standard(train_set)
qa_index = my_qa(10,train_set)
accu = acc(gold_index,qa_index)
print(accu)

0.5555555555555556


In [ ]:
#test on entire devtset
gold_index = gold_standard(dev_set)
qa_index = my_qa(63,dev_set)
acc = acc(gold_index,qa_index)
print(acc)